Về cơ bản của mạng nơ-ron thì chúng ta cần biết về:
- ANN (Artificial Neural Networks) và mối liên quan đến sinh học
- Ảnh hưởng của thuật toán Perceptron
- Thuật toán backpropagation và làm thế nào nó có thể sử dụng để train mạng nơ-ron multi-layer hiệu quả
- Làm sao để train mạng nơ-ron sử dụng thư viện Keras

## 10.1 Introduction to Neural Networks

Mạng nơ-ron là những khối xây dựng trong hệ thống Deep Learning. Theo đó để thành công trong Deep Learning, ta cần bắt đầu với những kiến thức cơ bản của mạng nơ-ron bao gồm: cấu trúc (architecture), nơ-ron( node types), và thuật toán cho việc học. Sau đó sẽ thảo luận về loại cấu trúc phổ biến nhất: **feedforward neural networks**<br>
Mạng ANN đơn giản là lấy cảm hứng từ não người. Mục đích của Deep Learning không phải bắt chước chức năng của não người, mà là lấy một phần trong đó cho phép chúng ta tạo ra những khuôn mẫu tương đồng cho công việc. Não người có xấp xỉ 10 tỷ nơ-ron và mỗi trong số chúng kết nối với khoảng 10 ngàn nơ-ron khác.

## 10.2 The Perceptron Algorithm

In [1]:
import numpy as np
class Perceptron:
    def __init__(self, N, alpha=0.1):
        # initialize the weight matrix and store the learning rate
        self.W = np.random.randn(N+1)/ np.sqrt(N)
        self.alpha = alpha
        
    def step(self, x):
        # apply the step function
        return 1 if x>0 else 0
    
    def fit(self, X, y, epochs=10):
        # insert a column of 1’s as the last entry in the feature
        # matrix -- this little trick allows us to treat the bias
        # as a trainable parameter within the weight matrix
        X = np.c_[X, np.ones((X.shape[0]))]
        
        # loop over the desired number of epochs
        for epoch in np.arange(0, epochs):
            # loop over each individual data point
            for (x, target) in zip(X, y):
                # take the dot product between the input features
                # and the weight matrix, then pass this value
                # through the step function to obtain the prediction
                p = self.step(np.dot(x, self.W))
                
                # only perform a weight update if our prediction
                # does not match the target
                if p!= target:
                    # determine the error
                    error = p - target
                    
                    # update the weight matrix
                    self.W += -self.alpha*error*x
    def predict(self, X, addBias=True):
        # ensure our input is a matrix
        X = np.atleast_2d(X)
        
        # check to see if the bias column should be added
        if addBias:
            # insert a column of 1’s as the last entry in the feature
            # matrix (bias)
            X = np.c_[X, np.ones((X.shape[0]))]
        
        # take the dot product between the input features and the
        # weight matrix, then pass the value through the step
        # function

        return  self.step(np.dot(X, self.W))

- Đầu tiên set ma trận trọng (dòng 5) số W ngẫu nhiên theo phân phối chuẩn, ma trận trọng số có N+1 entries. N là số input feature, 1 là cho bias. Sau đó chia cho căn bậc 2 của N để tỷ lệ lại (scale) ma trận trọng số -> dẫn đến nhanh hội tụ hơn.
- Định ra activation function. Ở đây là step function. S(x) > 0 đánh 1 , còn lại đánh 0.
- Lặp cho đủ số epoch định ra từ trước. Dòng 25 là hàm tính điểm (scoring function).
- Nếu p đúng với target thì thôi. Nếu không đúng thì đi qua Loss function (error). Từ đó cập nhật là ma trận trọng số W.
- Dòng 40 check dùng "bias trick" chưa, nếu không có gì thay đổi thì add thêm cột toàn số 1 ở entry cuối cùng của matrix

In [2]:
# construct the OR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [1]])

p = Perceptron(X.shape[1], alpha=0.1)
p.fit(X, y, epochs=20)

# now that our network is trained, loop over the data points
for (x, target) in zip(X, y):
    pred = p.predict(x)
    print("[INFO] data={}, ground-truth={}, pred={}".format(x, target[0], pred))

[INFO] data=[0 0], ground-truth=0, pred=0
[INFO] data=[0 1], ground-truth=1, pred=1
[INFO] data=[1 0], ground-truth=1, pred=1
[INFO] data=[1 1], ground-truth=1, pred=1


Đúng với dataset bitwise OR

In [3]:
# construct the AND dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [0], [0], [1]])

p = Perceptron(X.shape[1], alpha=0.1)
p.fit(X, y, epochs=20)

# now that our network is trained, loop over the data points
for (x, target) in zip(X, y):
    pred = p.predict(x)
    print("[INFO] data={}, ground-truth={}, pred={}".format(x, target[0], pred))

[INFO] data=[0 0], ground-truth=0, pred=0
[INFO] data=[0 1], ground-truth=0, pred=0
[INFO] data=[1 0], ground-truth=0, pred=0
[INFO] data=[1 1], ground-truth=1, pred=1


Đúng với dataset bitwise AND

In [4]:
# construct the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

p = Perceptron(X.shape[1], alpha=0.1)
p.fit(X, y, epochs=20)

# now that our network is trained, loop over the data points
for (x, target) in zip(X, y):
    pred = p.predict(x)
    print("[INFO] data={}, ground-truth={}, pred={}".format(x, target[0], pred))

[INFO] data=[0 0], ground-truth=0, pred=1
[INFO] data=[0 1], ground-truth=1, pred=0
[INFO] data=[1 0], ground-truth=1, pred=0
[INFO] data=[1 1], ground-truth=0, pred=0


Failed với dataset bitwise XOR. Với một single layer Perceptron thì không thể học được, cái ta cần là thêm nhiều layer.

## 10.3 Backpropagation and Multi-layer Networks

### 10.3.1 The Forward Pass

<img src="https://i.imgur.com/xamg60n.png" />

Ví dụ có một feature vector (0,1,1) , các initial weights ở Input layer, mạng 3x3x1. Để propagate values trong mạng và có được output phân loại cuối cùng, ta cần dot product giữa các input (X) và weights. Tiếp theo là dùng một activation fuction để ra output ở stage đó (activation trong trường hợp này là sigmoid function)<br>

Tính input của 3 node ở hidden layers<br><br>
Với hàm sigmoid là $\sigma(t) = 1/ (1+ e^{-t})$<br><br>
Node thứ nhất:<br> 
$\sigma((0 \times 0.351) + (1 \times 1.076) + (1 \times 1.116) )= 0.899$<br>
Node thứ hai:<br> 
$\sigma((0 \times -0.097)+ (1 \times -0.165)+ (1 \times 0.542)) = 0.593$<br>
Node thứ ba:<br> 
$\sigma((0 \times 0.457)+ (1 \times -0.165)+ (1 \times -0.331)) = 0.378$<br>

**Output của hidden layer** giờ sẽ là **input của layer cuối cùng** trong hình. Để tính toán, ta tiếp tục dot product và theo sau là sigmoid function<br>
Node ở layer cuối cùng:<br> 
$\sigma((0.899 \times 0.383)+ (0.593\times -0.327)+ (0.378 \times -0.329)) = 0.506$<br>

Vậy là output của network là 0.506. Ta định ra kết quả bằng bằng _sigmoid function_ (lớn hơn 0.5 đánh 1, còn lại đánh 0). Vậy là kết quả là 1. Tuy nhiên network không tự tin lắm vì kết quả này gần ngưỡng threshold. Lý tưởng là kết quả gần ở mức 0.98, 0.99

### 10.3.2 The Backward Pass

In [17]:
# import the necessary packages
import numpy as np

class NeuralNetwork:
    def __init__(self, layers, alpha=0.01):
        # initialize the list of weights matrices, then store the
        # network architecture and learning rate
        self.W = []
        self.layers = layers
        self.alpha = alpha
        
        # start looping from the index of the first layer but
        # stop before we reach the last two layers
        for i in np.arange(0, len(layers) -2):
            # randomly initialize a weight matrix connecting the
            # number of nodes in each respective layer together,
            # adding an extra node for the bias
            w = np.random.randn(layers[i]+1, layers[i+1] +1)
            self.W.append(w/ np.sqrt(layers[i]))
            
        # the last two layers are a special case where the input
        # connections need a bias term but the output does not
        w = np.random.randn(layers[-2]+1, layers[-1])
        self.W.append(w /np.sqrt(layers[-2]))
        
    def __repr__(self):
        # construct and return a string that represents the network
        # architecture
        return "NeuralNetwork: {}".format("-".join(str(l) for l in self.layers))
    
    def sigmoid(self, x):
        # compute and return the sigmoid activation value for a
        # given input value
        return 1/(1+ np.exp(-x))
    
    def sigmoid_deriv(self,x):
        # compute the derivative of the sigmoid function ASSUMING
        # that ‘x‘ has already been passed through the ‘sigmoid‘
        # function
        return x*(1-x)
    
    
    def fit(self, X, y, epochs=100, displayUpdate=1000):
        # insert a column of 1’s as the last entry in the feature
        # matrix -- this little trick allows us to treat the bias
        # as a trainable parameter within the weight matrix
        X = np.c_[X, np.ones((X.shape[0]))]
        
        # loop over the desired number of epochs
        for epoch in np.arange(0, epochs):
            # loop over each individual data point and train
            # our network on it
            for (x, target) in zip(X, y):
                self.fit_partial(x, target)
            
            # check to see if we should display a training update
            if epoch == 0 or (epoch +1) % displayUpdate == 0:
                loss = self.calculate_loss(X, y)
                print("[INFO] epoch={}, loss={}".format(epoch + 1, loss))
                
    def fit_partial(self, x, y):
        # construct our list of output activations for each layer
        # as our data point flows through the network; the first
        # activation is a special case -- it’s just the input
        # feature vector itself
        A = [np.atleast_2d(x)]
        
        # FEEDFORWARD:
        # loop over the layers in the network
        for layer in np.arange(0, len(self.W)):
            # feedforward the activation at the current layer by
            # taking the dot product between the activation and
            # the weight matrix -- this is called the "net input"
            # to the current layer
            net = A[layer].dot(self.W[layer])
            
            # computing the "net output" is simply applying our
            # nonlinear activation function to the net input
            out = self.sigmoid(net)
            
            # once we have the net output, add it to our list of
            # activations
            A.append(out)
            
        # BACKPROPAGATION
        # the first phase of backpropagation is to compute the
        # difference between our *prediction* (the final output
        # activation in the activations list) and the true target
        # value
        error= A[-1] -y
        # from here, we need to apply the chain rule and build our
        # list of deltas ‘D‘; the first entry in the deltas is
        # simply the error of the output layer times the derivative
        # of our activation function for the output value
        D = [error*self.sigmoid_deriv(A[-1])]
        
        # once you understand the chain rule it becomes super easy
        # to implement with a ‘for‘ loop -- simply loop over the
        # layers in reverse order (ignoring the last two since we
        # already have taken them into account)
        for layer in np.arange(len(A)-2, 0, -1):
            # the delta for the current layer is equal to the delta
            # of the *previous layer* dotted with the weight matrix
            # of the current layer, followed by multiplying the delta
            # by the derivative of the nonlinear activation function
            # for the activations of the current layer
            delta = D[-1].dot(self.W[layer].T)
            delta = delta* self.sigmoid_deriv(A[layer])
            D.append(delta)
            
        # since we looped over our layers in reverse order we need to
        # reverse the deltas
        D = D[::-1]
        # WEIGHT UPDATE PHASE
        # loop over the layers
        for layer in np.arange(0, len(self.W)):
            # update our weights by taking the dot product of the layer
            # activations with their respective deltas, then multiplying
            # this value by some small learning rate and adding to our
            # weight matrix -- this is where the actual "learning" takes
            # place
            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])

    def predict(self, X, addBias=True):
        # initialize the output prediction as the input features -- this
        # value will be (forward) propagated through the network to
        # obtain the final prediction
        p = np.atleast_2d(X)
        # check to see if the bias column should be added
        if addBias:
            # insert a column of 1’s as the last entry in the feature
            # matrix (bias)
            p = np.c_[p, np.ones((p.shape[0]))]
        # loop over our layers in the network
        for layer in np.arange(0, len(self.W)):
            # computing the output prediction is as simple as taking
            # the dot product between the current activation value ‘p‘
            # and the weight matrix associated with the current layer,
            # then passing this value through a nonlinear activation
            # function
            p = self.sigmoid(np.dot(p, self.W[layer]))
        # return the predicted value
        return p
    
    def calculate_loss(self, X, targets):
        # make predictions for the input data points then compute
        # the loss
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)
        # return the loss
        return loss

In [4]:
nn = NeuralNetwork([2, 2, 1])
print(nn)

NeuralNetwork: 2-2-1


In [18]:
# construct the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# define our 2-2-1 neural network and train it
nn = NeuralNetwork([2, 2, 1], alpha=0.5)
nn.fit(X, y, epochs=20000)

# now that our network is trained, loop over the XOR data points
for (x, target) in zip(X, y):
    # make a prediction on the data point and display the result
    # to our console
    pred = nn.predict(x)[0][0]
    step = 1 if pred > 0.5 else 0
    print("[INFO] data={}, ground-truth={}, pred={:.4f}, step={}".format(x, target[0], pred, step))

[INFO] epoch=1, loss=0.5645996425344315
[INFO] epoch=1000, loss=0.009872437593847162
[INFO] epoch=2000, loss=0.0029781163745660213
[INFO] epoch=3000, loss=0.0016944776613624837
[INFO] epoch=4000, loss=0.0011716338967807723
[INFO] epoch=5000, loss=0.0008909496790788585
[INFO] epoch=6000, loss=0.0007167575962894522
[INFO] epoch=7000, loss=0.0005984848314085457
[INFO] epoch=8000, loss=0.0005131003400104404
[INFO] epoch=9000, loss=0.0004486502936447409
[INFO] epoch=10000, loss=0.0003983273657738731
[INFO] epoch=11000, loss=0.0003579763393234865
[INFO] epoch=12000, loss=0.0003249201033097298
[INFO] epoch=13000, loss=0.0002973575966429816
[INFO] epoch=14000, loss=0.0002740333945841412
[INFO] epoch=15000, loss=0.00025404611125434066
[INFO] epoch=16000, loss=0.0002367321087929127
[INFO] epoch=17000, loss=0.00022159214443986434
[INFO] epoch=18000, loss=0.00020824355333352905
[INFO] epoch=19000, loss=0.00019638817691654194
[INFO] epoch=20000, loss=0.00018579030985804403
[INFO] data=[0 0], ground

Mô hình giờ đã tiên đoán đúng với dataset XOR